In [70]:
import pandas as pd
import numpy as np
from pathlib import Path

In [71]:
financial_features_df = pd.read_csv(Path('../Data/financial_features.csv'), infer_datetime_format=True)
news_features_df = pd.read_csv(Path('../Data/news_features.csv'), infer_datetime_format=True)
target_df = pd.read_csv(Path('../Data/target.csv'), infer_datetime_format=True)

In [83]:
model_data_df = target_df.merge(news_features_df, on=['date', 'symbol'], how='left').merge(financial_features_df, on=['date', 'symbol'], how='left')
model_data_df = model_data_df.set_index(['date'])
model_data_df.dropna().to_csv(Path('../Data/model_data.csv'))

In [73]:
model_data_df = model_data_df.reset_index()

time_span = pd.date_range(model_data_df['date'].iloc[0], model_data_df['date'].iloc[-1])
model_data_df = model_data_df.set_index(['date', 'symbol'])
for date in time_span:
    if ~model_data_df.index.isin([(pd.to_datetime(date), 'AZN')]).any():
        model_data_df.loc[(pd.to_datetime(date), 'AZN'), :] = None
    if ~model_data_df.index.isin([(pd.to_datetime(date), 'PFE')]).any():
        model_data_df.loc[(pd.to_datetime(date), 'PFE'), :] = None
    if ~model_data_df.index.isin([(pd.to_datetime(date), 'MRNA')]).any():
        model_data_df.loc[(pd.to_datetime(date), 'MRNA'), :] = None
    if ~model_data_df.index.isin([(pd.to_datetime(date), 'JNJ')]).any():
        model_data_df.loc[(pd.to_datetime(date), 'JNJ'), :] = None


In [74]:
model_data_df

close  compound_b  neg_b  neu_b  pos_b  \
date                symbol                                              
2020-05-18          AZN    -0.043409         NaN    NaN    NaN    NaN   
                    JNJ    -0.050989         NaN    NaN    NaN    NaN   
                    MRNA    6.896897         NaN    NaN    NaN    NaN   
                    PFE     0.217015         NaN    NaN    NaN    NaN   
2020-05-19          AZN     0.008815         NaN    NaN    NaN    NaN   
...                              ...         ...    ...    ...    ...   
2021-09-16 00:00:00 JNJ          NaN         NaN    NaN    NaN    NaN   
2021-09-17 00:00:00 AZN          NaN         NaN    NaN    NaN    NaN   
                    PFE          NaN         NaN    NaN    NaN    NaN   
                    MRNA         NaN         NaN    NaN    NaN    NaN   
                    JNJ          NaN         NaN    NaN    NaN    NaN   

                            compound_t  neg_t  neu_t  pos_t  lagged_returns  \
date                symbol                                                    
2020-05-18          AZN            NaN    NaN    NaN    NaN             NaN   
                    JNJ            NaN    NaN    NaN    NaN             NaN   
                    MRNA           NaN    NaN    NaN    NaN             NaN   
                    PFE            NaN    NaN    NaN    NaN             NaN   
2020-05-19          AZN            NaN    NaN    NaN    NaN             NaN   
...                                ...    ...    ...    ...             ...   
2021-09-16 00:00:00 JNJ            NaN    NaN    NaN    NaN             NaN   
2021-09-17 00:00:00 AZN            NaN    NaN    NaN    NaN             NaN   
                    PFE            NaN    NaN    NaN    NaN             NaN   
                    MRNA           NaN    NaN    NaN    NaN             NaN   
                    JNJ            NaN    NaN    NaN    NaN             NaN   

                            5_day_returns  volume_change  100_day_ma  \
date                symbol                                             
2020-05-18          AZN               NaN            NaN         NaN   
                    JNJ               NaN            NaN         NaN   
                    MRNA              NaN            NaN         NaN   
                    PFE               NaN            NaN         NaN   
2020-05-19          AZN               NaN            NaN         NaN   
...                                   ...            ...         ...   
2021-09-16 00:00:00 JNJ               NaN            NaN         NaN   
2021-09-17 00:00:00 AZN               NaN            NaN         NaN   
                    PFE               NaN            NaN         NaN   
                    MRNA              NaN            NaN         NaN   
                    JNJ               NaN            NaN         NaN   

                            50_day_ema  z_score  percentile_transformation  \
date                symbol                                                   
2020-05-18          AZN            NaN      NaN                        NaN   
                    JNJ            NaN      NaN                        NaN   
                    MRNA           NaN      NaN                        NaN   
                    PFE            NaN      NaN                        NaN   
2020-05-19          AZN            NaN      NaN                        NaN   
...                                ...      ...                        ...   
2021-09-16 00:00:00 JNJ            NaN      NaN                        NaN   
2021-09-17 00:00:00 AZN            NaN      NaN                        NaN   
                    PFE            NaN      NaN                        NaN   
                    MRNA           NaN      NaN                        NaN   
                    JNJ            NaN      NaN                        NaN   

                            stochastic_k  stochastic_d  
date                symbol                  

In [75]:
model_data_df['compound'] = model_data_df['compound_b'] + model_data_df['compound_t']
model_data_df['neg'] = model_data_df['neg_b'] + model_data_df['neg_t']
model_data_df['neu'] = model_data_df['neu_b'] + model_data_df['neu_t']
model_data_df['pos'] = model_data_df['pos_b'] + model_data_df['pos_t']
model_data_df = model_data_df.reset_index()
model_data_df = model_data_df.drop(columns=['compound_b', 'compound_t', 'neg_b', 'neg_t', 'neu_b', 'neu_t', 'pos_b', 'pos_t'])

In [76]:
model_data_df
model_data_df = model_data_df.set_index(['date']).fillna(model_data_df.mean())
X_f = model_data_df[['lagged_returns', '5_day_returns', 'volume_change', '100_day_ma', '50_day_ema', 'z_score', 'percentile_transformation', 'stochastic_k', 'stochastic_d']]
X_n = model_data_df[['compound', 'pos', 'neu', 'neg']]
y = model_data_df['close']

In [77]:
X_f

,lagged_returns,5_day_returns,volume_change,100_day_ma,50_day_ema,z_score,percentile_transformation,stochastic_k,stochastic_d
date,,,,,,,,,
2020-05-18,1.850372e-18,-2.220446e-17,-3.700743e-18,1.480297e-17,1.258253e-16,0.0,8.881784e-17,6.938894e-17,3.700743e-18
2020-05-18,1.850372e-18,-2.220446e-17,-3.700743e-18,1.480297e-17,1.258253e-16,0.0,8.881784e-17,6.938894e-17,3.700743e-18
2020-05-18,1.850372e-18,-2.220446e-17,-3.700743e-18,1.480297e-17,1.258253e-16,0.0,8.881784e-17,6.938894e-17,3.700743e-18
2020-05-18,1.850372e-18,-2.220446e-17,-3.700743e-18,1.480297e-17,1.258253e-16,0.0,8.881784e-17,6.938894e-17,3.700743e-18
2020-05-19,1.850372e-18,-2.220446e-17,-3.700743e-18,1.480297e-17,1.258253e-16,0.0,8.881784e-17,6.938894e-17,3.700743e-18
...,...,...,...,...,...,...,...,...,...
2021-09-16 00:00:00,1.850372e-18,-2.220446e-17,-3.700743e-18,1.480297e-17,1.258253e-16,0.0,8.881784e-17,6.938894e-17,3.700743e-18
2021-09-17 00:00:00,1.850372e-18,-2.220446e-17,-3.700743e-18,1.480297e-17,1.258253e-16,0.0,8.881784e-17,6.938894e-17,3.700743e-18
2021-09-17 00:00:00,1.850372e-18,-2.220446e-17,-3.700743e-18,1.480297e-17,1.258253e-16,0.0,8.881784e-17,6.938894e-17,3.700743e-18


In [78]:
from sklearn.base import clone
from sklearn.model_selection import train_test_split

def model_generator(X, y, algo):
    dates = X

    models = pd.Series(index=X.index)

    features = X
    target = y

    for date in dates:
        X_train, X_test, y_train, y_test = train_test_split(features, target)

        model = clone(algo)
        model.fit(X_train, y_train)
        models.loc[date] = model

    begin_dates = models.index
    end_dates = models.index[1:].append(pd.to_datetime(['2021-10-08']))

    predictions = pd.Series(index=features.index)
    for i, model in enumerate(models):
        X = features.xs(slice(begin_dates[i], end_dates[i]), level='date', drop_level=False)
        prediction = pd.Series(model.predict(X), index=X.index)
        predictions.loc[X.index] = prediction

    return models, predictions

In [79]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier

lr_models, lr_predictions = model_generator(X_f, y, algo=LinearRegression())
rf_models, rf_predictions = model_generator(X_n, y, algo=RandomForestClassifier())    

C:\Users\ayres\miniconda3\envs\machineLearning\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys
C:\Users\ayres\miniconda3\envs\machineLearning\lib\site-packages\ipykernel_launcher.py:22: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


AttributeError: 'Index' object has no attribute 'get_loc_level'

In [80]:
from sklearn.linear_model import LassoCV

def ensemble_data(X, y):
    X_with_y = X.join(y).replace({np.inf:None, -np.inf:None}).dropna()
    X = X_with_y.iloc[:,:-1]
    y = X_with_y.iloc[:,-1]
    return X, y

X_ensemble, y_ensemble = ensemble_data(X=pd.concat([lr_predictions.rename('linear'), rf_predictions.rename('forest')], axis=1,), y=y)

ensemble_models, ensemble_predictions = model_generator(X_ensemble, y_ensemble, LassoCV())

NameError: name 'lr_predictions' is not defined

In [ ]:
from sklearn.metrics import r2_score,mean_absolute_error

def calc_scorecard(y_pred,y_true):
    
    def make_df(y_pred,y_true):
        y_pred.name = 'y_pred'
        y_true.name = 'y_true'

        df = pd.concat([y_pred,y_true],axis=1).dropna()

        df['sign_pred'] = df.y_pred.apply(np.sign)
        df['sign_true'] = df.y_true.apply(np.sign)
        df['is_correct'] = 0
        df.loc[df.sign_pred * df.sign_true > 0 ,'is_correct'] = 1 # only registers 1 when prediction was made AND it was correct
        df['is_incorrect'] = 0
        df.loc[df.sign_pred * df.sign_true < 0,'is_incorrect'] = 1 # only registers 1 when prediction was made AND it was wrong
        df['is_predicted'] = df.is_correct + df.is_incorrect
        df['result'] = df.sign_pred * df.y_true 
        return df
    
    df = make_df(y_pred,y_true)
    
    scorecard = pd.Series()
    scorecard.loc['RSQ'] = r2_score(df.y_true,df.y_pred)
    scorecard.loc['MAE'] = mean_absolute_error(df.y_true,df.y_pred)
    scorecard.loc['edge'] = df.result.mean()
    scorecard.loc['noise'] = df.y_pred.diff().abs().mean()
    scorecard.loc['edge_to_noise'] = scorecard.loc['edge'] / scorecard.loc['noise']
    scorecard.loc['edge_to_mae'] = scorecard.loc['edge'] / scorecard.loc['MAE']
    return scorecard  

In [ ]:
pd.DataFrame([model.coef_ for model in ensemble_models], columns=X_ensemble.columns,index=ensemble_models.index).plot()

NameError: name 'ensemble_models' is not defined

In [ ]:
score_ens = calc_scorecard(y_pred=ensemble_predictions,y_true=y_ensemble).rename('LassoEnsemble')
score_linear = calc_scorecard(y_pred=lr_predictions,y_true=y_ensemble).rename('LinearRegression')
score_tree = calc_scorecard(y_pred=rf_predictions,y_true=y_ensemble).rename('RandomForest')


scores = pd.concat([score_linear,score_tree,score_ens],axis=1)
scores.loc['edge_to_noise'].plot.bar(color='blue',legend=True)
scores.loc['edge'].plot(color='red',legend=True)
scores.loc['noise'].plot(color='green',legend=True)

plt.show()
print(scores)